In [1]:
import os
import pandas as pd
from dotenv import load_dotenv
import google.generativeai as genai
load_dotenv() 

d:\SEM 7\ATS_Project\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
df_eda = pd.read_csv('TATAMOTORS.NS.csv', parse_dates=['Date'])
df_forecast = pd.read_csv('forecast_manual_vs_auto.csv')

def get_api_key() -> str:
    api_key = os.getenv('GOOGLE_API_KEY')
    if not api_key:
        raise RuntimeError(
            'Environment variable GOOGLE_API_KEY is not set. Create an API key in Google AI Studio and export it before running this notebook.'
        )
    return api_key

genai.configure(api_key=get_api_key())

def build_prompt(eda_df: pd.DataFrame, forecast_df: pd.DataFrame) -> str:
    prompt = f"""
You are a financial data analyst.
Below is a table with Tata Motors stock price forecasts using three ARIMA models (manual grid, manual intuition, auto).
Respond with the following:
- Summarize the main trends and differences in these forecast methods.
- Evaluate which method shows the most accurate and stable predictions on this time period.
- Reference data context from the historical EDA, especially regime changes or periods of volatility.
- Provide an executive summary fit for a stakeholder meeting.

Forecast Table:
{forecast_df.to_markdown(index=False)}

Historical Data Summary:
Mean: {eda_df['Close'].mean():.2f}
Median: {eda_df['Close'].median():.2f}
StdDev: {eda_df['Close'].std():.2f}
Min: {eda_df['Close'].min():.2f}
Max: {eda_df['Close'].max():.2f}
"""
    return prompt

def extract_response_text(response):
    aggregated_parts = []
    finish_reasons = []
    for candidate in getattr(response, 'candidates', []) or []:
        finish = getattr(candidate, 'finish_reason', None)
        finish_reasons.append(getattr(finish, 'name', str(finish)))
        content = getattr(candidate, 'content', None)
        if not content:
            continue
        for part in getattr(content, 'parts', []) or []:
            text = getattr(part, 'text', None)
            if text:
                aggregated_parts.append(text)
    return '\n'.join(aggregated_parts).strip(), finish_reasons

def prompt_feedback_message(response):
    feedback = getattr(response, 'prompt_feedback', None)
    return str(feedback) if feedback else ''

prompt = build_prompt(df_eda, df_forecast)

model = genai.GenerativeModel('gemini-2.5-flash')

response = model.generate_content(
    prompt,
    generation_config={
        'temperature': 0.5
    }
)

analysis_text, finish_reasons = extract_response_text(response)

print('=== Tata Motors Stock Forecast Analysis (Gemini Output) ===')
if analysis_text:
    print(analysis_text)
else:
    print('No text returned by the model.')
    if finish_reasons:
        print(f"Finish reasons: {', '.join(finish_reasons)}")
    feedback_message = prompt_feedback_message(response)
    if feedback_message:
        print(f'Prompt feedback: {feedback_message}')


=== Tata Motors Stock Forecast Analysis (Gemini Output) ===
As a financial data analyst, I've reviewed the Tata Motors stock price forecasts generated by three distinct ARIMA modeling approaches for September 2018.

### Summary of Main Trends and Differences in Forecast Methods

1.  **Manual Grid Search ARIMA(1, 1, 7):**
    *   **Trend:** This model projects a consistent, smooth, and gradual upward trend throughout September. It starts at 268.056 and ends at 269.302, representing an increase of approximately 1.25 points over the month.
    *   **Volatility:** It exhibits very low day-to-day volatility, with almost perfectly linear increases, suggesting a highly stable and predictable upward movement.

2.  **Manual Intuition ARIMA(3, 1, 3):**
    *   **Trend:** After an initial slight fluctuation, this model predicts an exceptionally flat trend. From September 7th onwards, the forecast remains constant at 267.811. It starts at 268.155 and ends at 267.811, showing a slight net decrease 